In [1]:
import tensorflow as tf

In [2]:
def net():
    return tf.keras.models.Sequential([
        # Here, we use a larger 11 x 11 window to capture objects. At the same
        # time, we use a stride of 4 to greatly reduce the height and width of
        # the output. Here, the number of output channels is much larger than
        # that in LeNet
        tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4,
                               activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        # Make the convolution window smaller, set padding to 2 for consistent
        # height and width across the input and output, and increase the
        # number of output channels
        tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same',
                               activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        # Use three successive convolutional layers and a smaller convolution
        # window. Except for the final convolutional layer, the number of
        # output channels is further increased. Pooling layers are not used to
        # reduce the height and width of input after the first two
        # convolutional layers
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                               activation='relu'),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                               activation='relu'),
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',
                               activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Flatten(),
        # Here, the number of outputs of the fully-connected layer is several
        # times larger than that in LeNet. Use the dropout layer to mitigate
        # overfitting
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Output layer. Since we are using Fashion-MNIST, the number of
        # classes is 10, instead of 1000 as in the paper
        tf.keras.layers.Dense(10)
    ])

In [3]:
X = tf.random.uniform((1, 224, 224, 1))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output_shape: \t', X.shape)

Conv2D output_shape: 	 (1, 54, 54, 96)
MaxPooling2D output_shape: 	 (1, 26, 26, 96)
Conv2D output_shape: 	 (1, 26, 26, 256)
MaxPooling2D output_shape: 	 (1, 12, 12, 256)
Conv2D output_shape: 	 (1, 12, 12, 384)
Conv2D output_shape: 	 (1, 12, 12, 384)
Conv2D output_shape: 	 (1, 12, 12, 256)
MaxPooling2D output_shape: 	 (1, 5, 5, 256)
Flatten output_shape: 	 (1, 6400)
Dense output_shape: 	 (1, 4096)
Dropout output_shape: 	 (1, 4096)
Dense output_shape: 	 (1, 4096)
Dropout output_shape: 	 (1, 4096)
Dense output_shape: 	 (1, 10)


In [4]:
class TrainCallback(tf.keras.callbacks.Callback):
    def __init__(self, net, train_iter, test_iter, num_epochs):
        self.net = net
        self.train_iter = train_iter
        self.test_iter = test_iter
        self.num_epochs = num_epochs

    def on_epoch_end(self, epoch, logs = None):
        test_acc = self.net.evaluate(self.test_iter, verbose = 0)
        metrics = (logs["loss"], logs["accuracy"], test_acc[1])
        print(f'epoch {epoch}, loss {metrics[0]:.3f}, train acc {metrics[1]:.3f}, 'f'test acc {metrics[2]:.3f}')

        if epoch == self.num_epochs - 1:
            print(f'loss {metrics[0]:.3f}, train acc {metrics[1]:.3f}, 'f'test acc {metrics[2]:.3f}')

In [5]:
def load_data_fashion_mnist(batch_size, resize=None):   #@save
    """Download the Fashion-MNIST dataset and then load it into memory."""
    mnist_train, mnist_test = tf.keras.datasets.fashion_mnist.load_data()
    # Divide all numbers by 255 so that all pixel values are between
    # 0 and 1, add a batch dimension at the last. And cast label to int32
    process = lambda X, y: (tf.expand_dims(X, axis=3) / 255,
                            tf.cast(y, dtype='int32'))
    resize_fn = lambda X, y: (
        tf.image.resize_with_pad(X, resize, resize) if resize else X, y)
    return (
        tf.data.Dataset.from_tensor_slices(process(*mnist_train)).batch(
            batch_size).shuffle(len(mnist_train[0])).map(resize_fn),
        tf.data.Dataset.from_tensor_slices(process(*mnist_test)).batch(
            batch_size).map(resize_fn))

In [6]:
batch_size = 128
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize = 224)

In [ ]:
lr, num_epochs = 0.9, 10
strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
with strategy.scope():
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    net_model = net()
    net_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
callback = TrainCallback(net_model, train_iter, test_iter, num_epochs)
net_model.fit(train_iter, epochs=num_epochs, verbose=0, callbacks=[callback])